<a href="https://colab.research.google.com/github/peterfarrell12/Spend_Classificatino_Model/blob/master/tool_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Modules**

In [0]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk import punkt
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
SENT_DETECTOR = nltk.data.load('tokenizers/punkt/english.pickle')
np.random.seed(500)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Upload Training File**

In [0]:
from google.colab import files

uploaded = files.upload()

Saving tooling_model.csv to tooling_model.csv


In [0]:
Corpus = pd.read_csv("tooling_model.csv")
Corpus.head()

,Item,Label
0,Hire of diesel generator,f
1,"Philmac Elbow polxpol 1""",f
2,Motor mount base,f
3,Repair of a pump and materials,f
4,Wam Vibrator motor type MVE300/3,f


**Preprocessing Of Text**

In [0]:
Corpus['Item'] = [str(i) for i in Corpus['Item']]
Corpus['Item'].dropna(inplace=True)
Corpus['Item'] = [entry.lower() for entry in Corpus['Item']]
Corpus['Item']= [word_tokenize(entry) for entry in Corpus['Item']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Item']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus.loc[index,'text_final'] = str(Final_words)

**Splitting Into Test/Train**

In [0]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Label'],test_size=0.3)


**Encoding Labels To 1 / 0**

In [0]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

**Creating Of Pipeline**

In [0]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('clf', svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'))])

**Fitting Of Model**

In [0]:
model = pipeline.fit(Train_X, Train_Y)

In [0]:
vectorizer = model.named_steps['vect']
clf = model.named_steps['clf']

In [0]:
print("accuracy score: " + str(model.score(Test_X, Test_Y)))

In [0]:
print(model.predict('screw'))

**Saving The Model To GDrive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model, path)